In [1]:
# Import Dependencies
import numpy as np
import pandas as pd
import datetime 
import math
import os 


In [2]:
#creating dataframes 
Nba_Salaries = 'Resources/NBA_season1718_salary.csv'
Team_boxscore = 'Resources/TeamBoxScore_2012-18.csv'
Standings =  'Resources/standings_2012-18.csv'
Boxscore =  'Resources/officialBoxScore_2012-18.csv'

ns = pd.read_csv(Nba_Salaries)
tb = pd.read_csv(Team_boxscore)
s = pd.read_csv(Standings)
bs = pd.read_csv(Boxscore)

In [3]:
ns.columns
tb.columns
s.columns
bs.columns

Index(['gmDate', 'gmTime', 'seasTyp', 'offLNm', 'offFNm', 'teamAbbr',
       'teamConf', 'teamDiv', 'teamLoc', 'teamRslt',
       ...
       'opptFIC40', 'opptOrtg', 'opptDrtg', 'opptEDiff', 'opptPlay%', 'opptAR',
       'opptAST/TO', 'opptSTL/TO', 'poss', 'pace'],
      dtype='object', length=119)

In [4]:
#count columns to get an idea of the count in each row
print(ns.count())
print(tb.count())
print(s.count())
print(bs.count())

Unnamed: 0     573
Player         573
Tm             573
season17_18    573
dtype: int64
gmDate        14758
gmTime        14758
seasTyp       14758
offLNm1       14758
offFNm1       14758
offLNm2       14758
offFNm2       14758
offLNm3       14754
offFNm3       14754
teamAbbr      14758
teamConf      14758
teamDiv       14758
teamLoc       14758
teamRslt      14758
teamMin       14758
teamDayOff    14758
teamPTS       14758
teamAST       14758
teamTO        14758
teamSTL       14758
teamBLK       14758
teamPF        14758
teamFGA       14758
teamFGM       14758
teamFG%       14758
team2PA       14758
team2PM       14758
team2P%       14758
team3PA       14758
team3PM       14758
              ...  
opptPTS1      14758
opptPTS2      14758
opptPTS3      14758
opptPTS4      14758
opptPTS5      14758
opptPTS6      14758
opptPTS7      14758
opptPTS8      14758
opptTREB%     14758
opptASST%     14758
opptTS%       14758
opptEFG%      14758
opptOREB%     14758
opptDREB%     14758
opptTO%    

In [5]:
ns = ns.drop('Unnamed: 0',axis=1)

In [6]:
ns.head()

,Player,Tm,season17_18
0,Stephen Curry,GSW,34682550.0
1,LeBron James,CLE,33285709.0
2,Paul Millsap,DEN,31269231.0
3,Gordon Hayward,BOS,29727900.0
4,Blake Griffin,DET,29512900.0


In [7]:
#created a new salary dataframe which sums total salary for teams
ns_draft = ns[["Tm","season17_18"]]
ns_final = ns_draft.rename(columns={"Tm":"Team", "season17_18":"salary"})

grouped_salaries = ns_final.groupby(['Team'])


salaries = grouped_salaries["salary"].sum()

salaries



Team
ATL    100217797.0
BOS    115009962.0
BRK     96039772.0
CHI     89425042.0
CHO    117228164.0
CLE    137288549.0
DAL     85821361.0
DEN    107889099.0
DET    120086105.0
GSW    137494845.0
HOU    119024081.0
IND     94429791.0
LAC    118907163.0
LAL    105355450.0
MEM    110104061.0
MIA    131222624.0
MIL    120686318.0
MIN    117468554.0
NOP    119725571.0
NYK    107855405.0
OKC    134294056.0
ORL     96925954.0
PHI    100794278.0
PHO     94818333.0
POR    119108924.0
SAC     95627528.0
SAS    116153554.0
TOR    116528796.0
UTA    107465014.0
WAS    124179842.0
Name: salary, dtype: float64

In [8]:
bs.head()

,gmDate,gmTime,seasTyp,offLNm,offFNm,teamAbbr,teamConf,teamDiv,teamLoc,teamRslt,...,opptFIC40,opptOrtg,opptDrtg,opptEDiff,opptPlay%,opptAR,opptAST/TO,opptSTL/TO,poss,pace
0,2012-10-30,19:00,Regular,Brothers,Tony,WAS,East,Southeast,Away,Loss,...,61.6667,105.6882,94.4447,11.2435,0.4390,16.7072,1.0476,33.3333,88.9409,88.9409
1,2012-10-30,19:00,Regular,Smith,Michael,WAS,East,Southeast,Away,Loss,...,61.6667,105.6882,94.4447,11.2435,0.4390,16.7072,1.0476,33.3333,88.9409,88.9409
2,2012-10-30,19:00,Regular,Workman,Haywoode,WAS,East,Southeast,Away,Loss,...,61.6667,105.6882,94.4447,11.2435,0.4390,16.7072,1.0476,33.3333,88.9409,88.9409
3,2012-10-30,19:00,Regular,Brothers,Tony,CLE,East,Central,Home,Win,...,56.0417,94.4447,105.6882,-11.2435,0.3765,18.8679,2.0000,84.6154,88.9409,88.9409
4,2012-10-30,19:00,Regular,Smith,Michael,CLE,East,Central,Home,Win,...,56.0417,94.4447,105.6882,-11.2435,0.3765,18.8679,2.0000,84.6154,88.9409,88.9409


In [47]:
bs_draft = bs[['teamAbbr','teamConf','teamDiv','gmDate']]
bs_draft2 = bs_draft.rename(columns={"teamAbbr":"Team","teamConf":"Conference","teamDiv":"Division","gmDate":"Date"})

bs_draft2.set_index('Team')
bss = bs_draft2[(bs_draft2['Date'] >= '2017-10-01')]

bs_final1 = bs_draft2[['Team','Conference','Division']]
bs_final = bs_final1.drop_duplicates()
bs_final

newTeam = []
team = bs_final['Team']

#loop through the team list, and keep track of which index we are looking at
for idx, val in enumerate(team):
    if val == 'SA':
        newTeam.append('SAS')
    elif val == 'GS':
        newTeam.append('GSW')
    elif val == 'NO':
        newTeam.append('NOP')
    elif val == 'NY':
        newTeam.append('NYK')
    else:
        newTeam.append(val)
bs_final['Team2'] = newTeam
bs_final = bs_final.drop('Team', axis=1).rename(columns={'Team2':'Team'})
bs_final

C:\Users\Elmer\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Conference,Division,Team
0,East,Southeast,WAS
3,East,Central,CLE
6,East,Atlantic,BOS
9,East,Southeast,MIA
12,West,Southwest,DAL
15,West,Pacific,LAL
18,West,Northwest,DEN
21,East,Atlantic,PHI
24,East,Central,IND
27,East,Atlantic,TOR


In [48]:
#took columns needed from the team boxscore table
tb_draft = tb[['teamAbbr','seasTyp','gmDate']]


tb_final = tb_draft.rename(columns={"teamAbbr":"Team","gmDate":"Date"})
tb_final.set_index ('Team')
tb_final


newTeam = []
team = tb_final['Team']

#loop through the team list, and keep track of which index we are looking at
for idx, val in enumerate(team):
    if val == 'SA':
        newTeam.append('SAS')
    elif val == 'GS':
        newTeam.append('GSW')
    elif val == 'NO':
        newTeam.append('NOP')
    elif val == 'NY':
        newTeam.append('NYK')
    else:
        newTeam.append(val)
tb_final['Team2'] = newTeam
tb_final = tb_final.drop('Team', axis=1).rename(columns={'Team2':'Team'})
tb_final



,seasTyp,Date,Team
0,Regular,2012-10-30,WAS
1,Regular,2012-10-30,CLE
2,Regular,2012-10-30,BOS
3,Regular,2012-10-30,MIA
4,Regular,2012-10-30,DAL
5,Regular,2012-10-30,LAL
6,Regular,2012-10-31,DEN
7,Regular,2012-10-31,PHI
8,Regular,2012-10-31,IND
9,Regular,2012-10-31,TOR


In [49]:
#grab all data for 2017-2018 season 
tb_filtered = tb_final[(tb_final['Date'] >= '2017-10-01')]
tb_filtered.head()

,seasTyp,Date,Team
12298,Regular,2017-10-17,BOS
12299,Regular,2017-10-17,CLE
12300,Regular,2017-10-17,HOU
12301,Regular,2017-10-17,GSW
12302,Regular,2017-10-18,CHA


In [50]:
# a look at the standings table
s.head()

,stDate,teamAbbr,rank,rankOrd,gameWon,gameLost,stk,stkType,stkTot,gameBack,...,rel%Indx,mov,srs,pw%,pyth%13.91,wpyth13.91,lpyth13.91,pyth%16.5,wpyth16.5,lpyth16.5
0,2012-10-30,ATL,3,3rd,0,0,-,-,0,0.5,...,0.0,0.0,0.0,0.500,0.0000,0.0000,82.0000,0.000,0.000,82.000
1,2012-10-30,BKN,3,3rd,0,0,-,-,0,0.5,...,0.0,0.0,0.0,0.500,0.0000,0.0000,82.0000,0.000,0.000,82.000
2,2012-10-30,BOS,14,14th,0,1,L1,loss,1,1.0,...,0.0,-13.0,-13.0,0.072,0.1687,13.8334,68.1666,0.131,10.742,71.258
3,2012-10-30,CHA,3,3rd,0,0,-,-,0,0.5,...,0.0,0.0,0.0,0.500,0.0000,0.0000,82.0000,0.000,0.000,82.000
4,2012-10-30,CHI,3,3rd,0,0,-,-,0,0.5,...,0.0,0.0,0.0,0.500,0.0000,0.0000,82.0000,0.000,0.000,82.000


In [51]:
#from the standings table take columns needed 
s_draft = s[['teamAbbr','rankOrd', 'stDate', 'gameWon','gameLost']]
#rename columns
s_final = s_draft.rename(columns={"teamAbbr":"Team","stDate":"Date"})
#grab data from 2017-2018 season
Standings = s_final[(s_final['Date'] >= '2017-10-01')]
Standings.head()

,Team,rankOrd,Date,gameWon,gameLost
24480,ATL,2nd,2017-10-17,0,0
24481,BKN,2nd,2017-10-17,0,0
24482,BOS,15th,2017-10-17,0,1
24483,CHA,2nd,2017-10-17,0,0
24484,CHI,2nd,2017-10-17,0,0


In [52]:
Ranking_draft = s_final[(s_final['Date'] > '2018-04-10')]
Ranking_draft1 = Ranking_draft.rename(columns={"rankOrd":"Team Rankings"})
Ranking = Ranking_draft1[['Team','Team Rankings']]
Ranking



#Creating an empty list which we will append the modified info to
newTeam = []
team = Ranking['Team']

#loop through the team list, and keep track of which index we are looking at
for idx, val in enumerate(team):
    if val == 'SA':
        newTeam.append('SAS')
    elif val == 'GS':
        newTeam.append('GSW')
    elif val == 'NO':
        newTeam.append('NOP')
    elif val == 'NY':
        newTeam.append('NYK')
    else:
        newTeam.append(val)
Ranking['Team2'] = newTeam
Ranking = Ranking.drop('Team', axis=1).rename(columns={'Team2':'Team'})
Ranking

C:\Users\Elmer\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Team Rankings,Team
29490,15th,ATL
29491,12th,BKN
29492,2nd,BOS
29493,10th,CHA
29494,13th,CHI
29495,4th,CLE
29496,13th,DAL
29497,9th,DEN
29498,9th,DET
29499,2nd,GSW


In [53]:
#created new columns for wins and losses
loss_draft = Standings[['Team','gameLost']]
won_draft = Standings[['Team','gameWon']]

#created new measure that counted the total amount in the row 

Team_Losses = loss_draft[loss_draft.groupby('Team').cumcount()==167]
Team_Wins = won_draft[won_draft.groupby('Team').cumcount()==167]


Team_Losses.head()


,Team,gameLost
29490,ATL,58
29491,BKN,54
29492,BOS,27
29493,CHA,46
29494,CHI,55


In [54]:
#merge wins, losses

winsLosses =  pd.merge(Team_Losses, Team_Wins, on="Team", how="inner")
winsLosses

#Creating an empty list which we will append the modified info to
newTeam = []
team = winsLosses['Team']

#loop through the team list, and keep track of which index we are looking at
for idx, val in enumerate(team):
    if val == 'SA':
        newTeam.append('SAS')
    elif val == 'GS':
        newTeam.append('GSW')
    elif val == 'NO':
        newTeam.append('NOP')
    elif val == 'NY':
        newTeam.append('NYK')
    else:
        newTeam.append(val)
winsLosses['Team2'] = newTeam
winsLosses = winsLosses.drop('Team', axis=1).rename(columns={'Team2':'Team', 'gameLost':'Losses', 'gameWon':'Wins'})
winsLosses = winsLosses[['Team','Wins','Losses']]
winsLosses

,Team,Wins,Losses
0,ATL,24,58
1,BKN,28,54
2,BOS,55,27
3,CHA,36,46
4,CHI,27,55
5,CLE,50,32
6,DAL,24,58
7,DEN,46,36
8,DET,39,43
9,GSW,58,24


In [55]:
#merge salaries to Wins_losses
Team_draft = pd.merge(winsLosses, salaries, on="Team")
Team_draft

,Team,Wins,Losses,salary
0,ATL,24,58,100217797.0
1,BOS,55,27,115009962.0
2,CHI,27,55,89425042.0
3,CLE,50,32,137288549.0
4,DAL,24,58,85821361.0
5,DEN,46,36,107889099.0
6,DET,39,43,120086105.0
7,GSW,58,24,137494845.0
8,HOU,65,17,119024081.0
9,IND,48,34,94429791.0


In [56]:
team = pd.merge(Team_draft, Ranking, on="Team", how="inner")
team

,Team,Wins,Losses,salary,Team Rankings
0,ATL,24,58,100217797.0,15th
1,BOS,55,27,115009962.0,2nd
2,CHI,27,55,89425042.0,13th
3,CLE,50,32,137288549.0,4th
4,DAL,24,58,85821361.0,13th
5,DEN,46,36,107889099.0,9th
6,DET,39,43,120086105.0,9th
7,GSW,58,24,137494845.0,2nd
8,HOU,65,17,119024081.0,1st
9,IND,48,34,94429791.0,5th


In [57]:
#merging data frames 

NBA_2017_2018_raw = pd.merge(team, bs_final, on="Team", how="inner" )
NBA_2017_2018_data = NBA_2017_2018_raw.sort_values(by=[ 'Conference','Team Rankings'])
NBA_2017_2018_data.set_index ('Team')
NBA_2017_2018_data



,Team,Wins,Losses,salary,Team Rankings,Conference,Division
17,NYK,29,53,107855405.0,11th,East,Atlantic
2,CHI,27,55,89425042.0,13th,East,Central
19,ORL,25,57,96925954.0,14th,East,Southeast
0,ATL,24,58,100217797.0,15th,East,Southeast
25,TOR,59,23,116528796.0,1st,East,Atlantic
1,BOS,55,27,115009962.0,2nd,East,Atlantic
20,PHI,52,30,100794278.0,3rd,East,Atlantic
3,CLE,50,32,137288549.0,4th,East,Central
9,IND,48,34,94429791.0,5th,East,Central
13,MIA,44,38,131222624.0,6th,East,Southeast


In [60]:
NBA_2017_2018_data.describe

<bound method NDFrame.describe of    Team  Wins  Losses       salary Team Rankings Conference   Division
17  NYK    29      53  107855405.0          11th       East   Atlantic
2   CHI    27      55   89425042.0          13th       East    Central
19  ORL    25      57   96925954.0          14th       East  Southeast
0   ATL    24      58  100217797.0          15th       East  Southeast
25  TOR    59      23  116528796.0           1st       East   Atlantic
1   BOS    55      27  115009962.0           2nd       East   Atlantic
20  PHI    52      30  100794278.0           3rd       East   Atlantic
3   CLE    50      32  137288549.0           4th       East    Central
9   IND    48      34   94429791.0           5th       East    Central
13  MIA    44      38  131222624.0           6th       East  Southeast
14  MIL    44      38  120686318.0           6th       East    Central
27  WAS    43      39  124179842.0           8th       East  Southeast
6   DET    39      43  120086105.0         

In [61]:
NBA_2017_2018_data.to_csv('NBA_2017_2018_data.csv', encoding='utf-8')